In [1]:
%run ../../main.py

In [2]:
from pyarc.algorithms import top_rules
from pyarc.data_structures import TransactionDB, Consequent, Antecedent, Item, ClassAssocationRule
from pyarc.algorithms import M1Algorithm, M2Algorithm, createCARs
import pandas as pd
from sklearn.metrics import accuracy_score

In [3]:
data_train = pd.read_csv("c:/code/python/machine_learning/assoc_rules/train/lymph0.csv")
data_test = pd.read_csv("c:/code/python/machine_learning/assoc_rules/test/lymph0.csv")

txns_train = TransactionDB.from_DataFrame(data_train)
txns_test = TransactionDB.from_DataFrame(data_train)

isinstance(txns_test, TransactionDB)



True

In [4]:
from pyarc import CBA, TransactionDB

data_train = pd.read_csv("c:/code/python/machine_learning/assoc_rules/train/lymph0.csv")
data_test = pd.read_csv("c:/code/python/machine_learning/assoc_rules/test/lymph0.csv")

txns_train = TransactionDB.from_DataFrame(data_train)
txns_test = TransactionDB.from_DataFrame(data_train)


# get the best association rules
rules = top_rules(txns_train.string_representation)

# convert them to class association rules
cars = createCARs(rules)

classifier = M1Algorithm(cars, txns_train).build()

accuracy = classifier.test_transactions(txns_test)

accuracy

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=19
Rule count: 40504, Iteration: 1
Target rule count satisfied: 1000


0.9615384615384616

In [5]:


directory = "c:/code/python/machine_learning/assoc_rules"

def func(datasetname, unique_transactions=True):
    pd_ds = pd.read_csv("c:/code/python/machine_learning/assoc_rules/train/{}.csv".format(datasetname))
    txns = TransactionDB.from_DataFrame(pd_ds, unique_transactions=unique_transactions)
    
    txns_test = TransactionDB.from_DataFrame(pd.read_csv("c:/code/python/machine_learning/assoc_rules/test/{}.csv".format(datasetname)))

    rules = top_rules(txns.string_representation, appearance=txns.appeardict)

    rules.sort(reverse=True)


    cars = createCARs(rules)

    cars.sort(reverse=True)

    if len(cars) > 1000:
        cars = cars[:1000]
        

    print("len(rules)", len(cars))

    m1 = M1Algorithm(cars, txns)
    
    m2 = M2Algorithm(cars, txns)
    
    m1clf = m1.build()
    m2clf = m2.build()
    
    
    actual = list(map(lambda i: i.value, txns_test.class_labels))

    pred = m1clf.predict_all(txns_test)
    predM2 = m2clf.predict_all(txns_test)
    
    accM2 = accuracy_score(predM2, actual)
    acc = accuracy_score(pred, actual)

    return acc, accM2



def mean_func(dataset_name, start=0, end=10, unique_transactions=True):
    files = [ dataset_name + repr(i) for i in range(start, end) ]

    accs = []
    accsM2 = []
    
    for file in files:
        acc, accM2 = func(file, unique_transactions=unique_transactions)
        print("done", file, acc)
        print("done m2", file, accM2)
        accs.append(acc)
        accsM2.append(accM2)
        
    mn = sum(accs) / len(accs)
    mnM2 = sum(accsM2) / len(accsM2)
    
    return mn, mnM2


                
datasets = [
    "iris",
    "breast-w",
    "anneal",
    "hypothyroid",
    "ionosphere",
    "lymph",
    "vehicle",
    "autos",
    "diabetes",
    "glass",
    "heart-h",
    "tic-tac-toe",
    "australian",
    "sick",
    "segment",
    "spambase",
    "sonar",
    "vowel",
    "hepatitis",
    "credit-a",
    "mushroom",
    "house-votes-84",
    "soybean",
    "primary-tumor",
    "credit-g",
    "audiology",
    "breast-cancer",
    "balance-scale",
    "heart-c",
    "kr-vs-kp",
    "pima",
    "heart-statlog"]    

means = []
meansM2 = []
for dataset in ["anneal"]:
    acc, accM2 = mean_func(dataset)
    print("*****")
    print("M1", dataset, acc)
    print("M2", dataset, accM2)
    print("******")
    
    means.append((dataset, acc))
    meansM2.append((dataset, accM2))
    
print("M1")
print(means)
print("\nM2")
print(meansM2)

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=33
Rule count: 6309, Iteration: 1
Target rule count satisfied: 1000
len(rules) 1000
done anneal0 1.0
done m2 anneal0 1.0
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=33
Rule count: 6951, Iteration: 1
Target rule count satisfied: 1000
len(rules) 1000
done anneal1 1.0
done m2 anneal1 1.0
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=33
Rule count: 6535, Iteration: 1
Target rule count satisfied: 1000
len(rules) 1000
done anneal2 1.0
done m2 anneal2 1.0
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=33
Rule count: 6821, Iteration: 1
Target rule count satisfied: 1000
len(rules) 1000
done anneal3 0.989010989011
done m2 anneal3 0.989010989011
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=33
Rule count: 6745, Iteration: 1


KeyboardInterrupt: 

In [7]:

m2df = pd.DataFrame(dict(map(lambda i:  (i[0], [i[1]]), meansM2))).transpose()
m2df.columns = ["m2"]

m1df = pd.DataFrame(dict(map(lambda i:  (i[0], [i[1]]), means))).transpose()
m1df.columns = ["m1"]


acc_df = m1df.join(m2df)

acc_df.to_csv("../data/accuracies.csv")

#acc_df = pd.read_csv("../data/accuracies.csv")



In [8]:
acc_df

,m1,m2
anneal,0.991061,0.991061
audiology,0.455563,0.459911
australian,0.859381,0.859381
autos,0.809592,0.809592
balance-scale,0.740807,0.739219
breast-cancer,0.706708,0.706708
breast-w,0.951343,0.951343
credit-a,0.831947,0.831947
credit-g,0.733000,0.733000
diabetes,0.743558,0.743558


In [9]:
ARC_accs = pd.read_csv("../data/arc_accs_bugfixed.csv")

ARC_accs_mod = ARC_accs[["dataset", "accuracy"]].set_index("dataset")

ARC_accs_mod.columns = ["ARC"]

accs = acc_df.join(ARC_accs_mod)

print("M1", accs["m1"].mean())
print("M2", accs["m2"].mean())
print("ARC", accs["ARC"].mean())

M1 0.8105017932631686
M2 0.8092027413359664
ARC 0.7828125


In [ ]:
ARC_accs = pd.read_csv("../data/arc_accs3.csv")

ARC_accs_mod = ARC_accs[["dataset", "accuracy"]].set_index("dataset")

ARC_accs_mod.columns = ["ARC"]

accs = acc_df.join(ARC_accs_mod)

print("M1", accs["m1"].mean())
print("M2", accs["m2"].mean())
print("ARC", accs["ARC"].mean())

In [ ]:
acc_df

In [ ]:
ARC_accs = pd.read_csv("../data/arc_accs3.csv")

ARC_accs_mod = ARC_accs[["dataset", "accuracy"]].set_index("dataset")

ARC_accs_mod.columns = ["ARC"]

accs = acc_df.join(ARC_accs_mod)

print("M1", accs["m1"].mean())
print("M2", accs["m2"].mean())
print("ARC", accs["ARC"].mean())

In [ ]:
ARC_accs = pd.read_csv("../data/arc_accs_bugfixed.csv")

ARC_accs["accuracy"].mean()

In [ ]:
printaccs = accs[["m1", "m2", "ARC"]] * 100
printaccs = printaccs[["m1", "m2", "ARC"]].round(1)

In [ ]:
printaccs.to_csv("../data/all_accuracies.csv")